### 0.1 Case Study

#### Scenario
At BMW, we reimagine the future of mobility. Lets fast forward to 2030, and flying taxis are roaming above our cities, bringing people to
their desired destination. You work for, Duoro Hawk a company that is pioneering the world's first large fleet of fully electric, self-piloting
autonomous flying taxis. The company wants to deploy the first network of autonomous air taxis in the coming year. As part of our data
science and enginering team, you are responsible for predicting the destination of our fleet of autonomous flying taxis based on the
manned test flights that have been performed.

#### About the Dataset
A fictional dataset describing a complete year (from 01/07/2014 to 30/06/2014) of all the trajectories for all 442 of our flying taxis that
were simulated in the city of Porto. Our autonomous fleet of taxis fly from a central ground station
• There are three different types of rides: A) phone call-based, B) stand-based where people wait at a stand for their flying taxi or C) 
random place. For type A, we provide an anonymized ID, to represent the telephone call. Categories B and C refers to cases where the
taxis were directly called by the customer.

#### Dataset
##### train.csv
Each data sample corresponds to one completed trip. It contains a total of 9 (nine) features, described as follows:

- TRIP_ID: (String) It contains an unique identifier for each trip;

- CALL_TYPE: (char) It identifies the way used to demand this service. It may contain one of three possible values:
     - ‘A’ if this trip was dispatched from the central;
     - ‘B’ if this trip was demanded directly to a taxi driver on a specific stand;
     - ‘C’ otherwise (i.e. a trip demanded on a random street).
     
- ORIGIN_CALL: (integer) It contains an unique identifier for each phone number which was used to demand, at least, one service. It identifies the trip’s customer if CALLTYPE=’A’. Otherwise, it assumes a NULL value;

- ORIGINSTAND: (integer): It contains an unique identifier for the taxi stand. It identifies the starting point of the trip if CALLTYPE=’B’. Otherwise, it assumes a NULL value;

- WEATHER: (String): Information on the weather that day, unique values include: Sunny, Rainy, Cloudy, Windy, and Foggy
- TAXI_ID: (integer): It contains an unique identifier for the flying taxi that performed each trip;
- TIMESTAMP: (integer) Unix Timestamp (in seconds). It identifies the trip’s start;
- MISSING_DATA: (Boolean) It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing
- POLYLINE: (String): It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as
- [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The last list item corresponds to the trip’s destination while the first one represents its start


##### test.csv
Personal records for the remaining one-third (~110) of the trips, to be used as test data. Your task is to predict the value of coordinates of the trip‘s destination

##### sample_submission.csv 
A submission file in the correct format.
- TripId - Id for each Tip in the test set
- Longitude - the longitude of the destination of the flying taxi
- Latitude – the latitude of the destination of the flying taxi

The total travel time of the trip (the prediction target of this competition) is defined as the (number of points-1) x 15 seconds. For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.

### 0.2 Imports

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

Matplotlib is building the font cache; this may take a moment.


In [2]:
def print_sample_size(data, add_text=''):
    print(f'{add_text} Size data samples {data.shape[0]}')

### 0.3 Load Data

#### (!) Due to multiple issues caused by the size of the train_df file, all files will be uploaded to a S3 location as parquet files and loaded from there. 

In [3]:
train_data = pd.read_csv('s3://think-tank-casestudy/raw_data/train_df.csv', header=0, index_col=False)
train_data.to_parquet('s3://think-tank-casestudy/raw_data/train_data.parquet')

test_data = pd.read_csv('data/test_df.csv',header=0, index_col=False)
test_data.to_parquet('s3://think-tank-casestudy/raw_data/test_data.parquet')

In [4]:
train_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/train_data.parquet')
test_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/test_data.parquet')

In [5]:
sample_submission = pd.read_csv('data/sampleSubmission.csv', header=0, index_col=False)

In [6]:
sample_submission.head()

Target prediction is longitude and latitude for each TRIP in test_data --> **MULTI REGRESSION OUTPUT**
Relevant for modelling later on

In [7]:
train_data.DAY_TYPE.value_counts()

A    1710670
Name: DAY_TYPE, dtype: int64

In [8]:
test_data.DAY_TYPE.value_counts()

A    320
Name: DAY_TYPE, dtype: int64

DAY_TYPE is not mentioned in list of attributes and attribute is uniformly distributed for both train and test data --> no information gain and can therefore be dropped

In [9]:
train_data = train_data.copy().drop(train_data.columns[0],axis=1).drop(['DAY_TYPE'],axis=1)
test_data = test_data.copy().drop(['DAY_TYPE'], axis=1)

In [10]:
def adjust_datatypes(data):
    """
    adjust to appropriate datatypes
    """
    data.TRIP_ID = data.TRIP_ID.astype(object)
    data.CALL_TYPE = data.CALL_TYPE.astype(object)
    data.ORIGIN_STAND = data.ORIGIN_STAND.astype(object)
    data.ORIGIN_CALL = data.ORIGIN_CALL.astype(object)
    data.TAXI_ID = data.TAXI_ID.astype(object)
    data['TIMESTAMP_DT'] = data['TIMESTAMP'].apply(lambda value_unix: 
                                                   datetime.fromtimestamp(value_unix).strftime('%Y-%m-%d %H:%M:%S'))
    data.TIMESTAMP_DT = pd.to_datetime(data.TIMESTAMP_DT)
    data['POLYLINE_LIST'] = data['POLYLINE'].apply(lambda value: json.loads(value))
    return data

In [11]:
train_data = adjust_datatypes(train_data.copy())
test_data = adjust_datatypes(test_data.copy())

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   TRIP_ID        object        
 1   CALL_TYPE      object        
 2   ORIGIN_CALL    object        
 3   ORIGIN_STAND   object        
 4   TAXI_ID        object        
 5   TIMESTAMP      int64         
 6   MISSING_DATA   bool          
 7   POLYLINE       object        
 8   WEATHER        object        
 9   TIMESTAMP_DT   datetime64[ns]
 10  POLYLINE_LIST  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(8)
memory usage: 132.1+ MB


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TRIP_ID        320 non-null    object        
 1   CALL_TYPE      320 non-null    object        
 2   ORIGIN_CALL    72 non-null     object        
 3   ORIGIN_STAND   123 non-null    object        
 4   TAXI_ID        320 non-null    object        
 5   TIMESTAMP      320 non-null    int64         
 6   MISSING_DATA   320 non-null    bool          
 7   POLYLINE       320 non-null    object        
 8   WEATHER        320 non-null    object        
 9   TIMESTAMP_DT   320 non-null    datetime64[ns]
 10  POLYLINE_LIST  320 non-null    object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(8)
memory usage: 25.4+ KB


In [14]:
print(train_data.TIMESTAMP_DT.min())
print(train_data.TIMESTAMP_DT.max())
print(test_data.TIMESTAMP_DT.min())
print(test_data.TIMESTAMP_DT.max())

2013-07-01 00:00:53
2014-06-30 23:59:56
2014-08-14 16:02:23
2014-12-21 14:29:19


All dates are in previously defined valid ranges

### 0.4 Sanity Checks

In [15]:
#Assert that train and test data have same column shape and attributes
def perform_sanity_checks():
    try:
        assert(train_data.shape[1] == test_data.shape[1])
        print("Column shape train vs test passed")
        assert((train_data.columns == test_data.columns).all())
        print("Column naming train vs test passed")
        assert(train_data.TRIP_ID.nunique() == train_data.shape[0])
        print("Check for unique trips passed - train data")
        assert(test_data.TRIP_ID.nunique() == test_data.shape[0])
        print("Check for unique trips passed - test data")
        print('All checks passed!')
    except:
        print("Sanity Check failed")

In [16]:
perform_sanity_checks()

Column shape train vs test passed
Column naming train vs test passed
Sanity Check failed


### 0.5 Cleaning Data

#### 0.5.1 NAN/Null Missing values

In [17]:
train_data.isnull().sum()

TRIP_ID                0
CALL_TYPE              0
ORIGIN_CALL      1345900
ORIGIN_STAND      904091
TAXI_ID                0
TIMESTAMP              0
MISSING_DATA           0
POLYLINE               0
WEATHER                0
TIMESTAMP_DT           0
POLYLINE_LIST          0
dtype: int64

In [18]:
test_data.isnull().sum()

TRIP_ID            0
CALL_TYPE          0
ORIGIN_CALL      248
ORIGIN_STAND     197
TAXI_ID            0
TIMESTAMP          0
MISSING_DATA       0
POLYLINE           0
WEATHER            0
TIMESTAMP_DT       0
POLYLINE_LIST      0
dtype: int64

ORIGIN_CALL and ORIGIN_STAND have null values which is to be expected as they are determined dependent on the call type

#### 0.5.2 Duplicated TRIP_IDs

The Sanity Checks in 0.4 showed that the TRIP_IDs are not unique. 

In [19]:
vc = train_data.TRIP_ID.value_counts().reset_index()
DUPLICATED_IDs = vc[vc.TRIP_ID > 1]['index'].unique()
print(f'{len(DUPLICATED_IDs)} TRIP_IDs are duplicated')
print(f'{(len(DUPLICATED_IDs)/train_data.TRIP_ID.nunique()*100)} % out of all unique TRIPs.')

80 TRIP_IDs are duplicated
0.0046767516919610725 % out of all unique TRIPs.


**Findings**:
- 159 cases
- Missing Data == False for all cases
- 80 TRIP_IDs are duplicated
- Affected data is insignifcant (less than 1% of all  TRIPs)

**Assumptions**:
- Potential reasons could be cancellation by dispatcher after a person called for some reasons, failed flight attempts, broken flight taxi etc.
- The trips per ID with the longest POLYLINE are kept as these are assumed to be valid trips. Additionally trips with no POLYLINE or only one coordinate point are assumed invalid and filtered from the dataset. 
- Also it is assumed that only POLYLINEs with at least 5 coordinate points are sufficient. 
- Further investigation will should be done and analyzed together with sensor/technical data from flight taxi. Also the reason could be that a flight is interrupted and re-started again, that could be analyzed by plotting the POLYLINE and compare the start and end point of the duplicated TRIPs. Will be part of further optimization

To do as mentioned in Assumptions, the number of points in the POLYLINE needs to be calculated. In addition we calculate the total flight time at this point.

#### 0.5.3 Data Cleaning POLYLINE
To do cleaning regarding the POLYLINE, a few more attributes are calculated:
- N_COORDINATE_POINTS - number of total points
- TOTAL_FLIGHT_TIME_SECONDS, TOTAL_FLIGHT_TIME_MINUTES - flight time total
- START_POINT - Starting point for each trip
- DEST_POINT - Last point for each trip 
- TOTAL_DISTANCE - total distance of trip in km with haversine formula

In [20]:
def calculate_POLYLINE_features(data):
    """
    calculates length of POLYLINE as N_COORDINATE_POINTS
    calculates total flight time in seconds and minutes as TOTAL_FLIGHT_TIME_SECONDS and TOTAL_FLIGHT_TIME_MINUTES
    """
    data['N_COORDINATE_POINTS'] = data['POLYLINE_LIST'].apply(lambda value: len(value))
    #total flight time
    data['TOTAL_FLIGHT_TIME_SECONDS'] = data.apply(lambda row: (row.N_COORDINATE_POINTS-1)*15, axis=1)
    data['TOTAL_FLIGHT_TIME_MINUTES'] = data.TOTAL_FLIGHT_TIME_SECONDS / 60
    return data

def calculate_total_distance(data):
    data['START_POINT'] = data['POLYLINE_LIST'].apply(lambda value: value[0])
    data['DEST_POINT'] = data['POLYLINE_LIST'].apply(lambda value: value[-1])   
    data['TOTAL_DISTANCE_KM'] = data.apply(lambda row:
                                       haversine_distance(lat1=row.START_POINT[1],
                                                lat2=row.DEST_POINT[1],
                                                lon1=row.START_POINT[0],
                                                lon2=row.DEST_POINT[0])
                                        ,axis=1
                                       )
    return data

def haversine_distance(lat1, lat2, lon1, lon2):
    #analog to following source -> https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html
    from sklearn.metrics.pairwise import haversine_distances
    from math import radians
    point_A = [lon1, lat1] 
    point_B = [lon2, lat2]
    pointA_in_radians = [radians(_) for _ in point_A]
    pointB_in_radians = [radians(_) for _ in point_B]
    result = haversine_distances([pointA_in_radians, pointB_in_radians])
    result = result * 6371000/1000  
    return result[0][1]

def filter_invalid_trips(data):
    """
    filters trips with less than 5 coordinate points and takes data sample with longest POLYLINE for duplicated TRIP IDs
    """
    data = data[data.N_COORDINATE_POINTS >= 5]
    vc = data.TRIP_ID.value_counts().reset_index()
    DUPLICATES_IDs = vc[vc.TRIP_ID > 1]['index'].unique()
    if len(DUPLICATES_IDs) > 0:
        data_duplicated = data[data.TRIP_ID.isin(DUPLICATES_IDs)]
        data_valid = data[~data.TRIP_ID.isin(DUPLICATES_IDs)]
        data_filtered = data_duplicated.groupby('TRIP_ID').apply(lambda datachunk: datachunk[datachunk.N_COORDINATE_POINTS == datachunk.N_COORDINATE_POINTS.max()])
        data = pd.concat([data_filtered,data_valid],axis=0)
    return data

In [21]:
train_data = calculate_POLYLINE_features(train_data.copy())
test_data = calculate_POLYLINE_features(test_data.copy())

**Filter invalid trips**
- The trips per ID with the longest POLYLINE are kept as these are assumed to be valid trips. Additionally trips with no POLYLINE or only one coordinate point are assumed invalid and filtered from the dataset.

In [22]:
print_sample_size(train_data, 'Including invalid trips -')
print_sample_size(test_data, 'Including invalid trips -')
train_data_filtered = filter_invalid_trips(train_data.copy())
test_data_filtered = filter_invalid_trips(test_data.copy())
print_sample_size(train_data_filtered, 'Excluding invalid trips - ')
print_sample_size(test_data_filtered, 'Excluding invalid trips - ')

Including invalid trips - Size data samples 1710670
Including invalid trips - Size data samples 320
Excluding invalid trips -  Size data samples 1658565
Excluding invalid trips -  Size data samples 299


In [23]:
train_data_filtered = train_data_filtered.reset_index(drop=True)
test_data_filtered = test_data_filtered.reset_index(drop=True)

In [ ]:
train_data_filtered = calculate_total_distance(train_data_filtered.copy())
test_data_filtered = calculate_total_distance(test_data_filtered.copy())

#### 0.5.4 MISSING_DATA == TRUE

In [ ]:
print(f'Train Data: {train_data_filtered.MISSING_DATA.value_counts()[1]} Trips with MISSING_DATA == True')

In [ ]:
test_data_filtered.MISSING_DATA.value_counts()

In [ ]:
train_data_filtered[train_data_filtered.MISSING_DATA == True]

- Amount of data with missing values insignificant compared to total amount of data
- Majority of trips is WEATHER == Rainy, however total amount of trips is not significant enought to draw a conclusion/make an assumption
- Number of points/length of polyline is in general unequal so there is no indication in that sense how much data is missing, also no information if data is missing at the start/middle or end of POLYLINE 

Based on these Findings, I would simply drop these values, mainly as their effect is expected to be very little. If the number of data samples would be higher, I would try to impute the missing coordinates in this case with the Nearest Neighbour. However the problem of knowing if the cooordinates are missing in start/middle/end would prevail. In case I find very similar trips through additional logic (difference in n_coordinate_points <= 5 and overall_distance between points < threshold) I could minimize this problem. These tasks could be part of further optimization.

In [ ]:
train_data_filtered = train_data_filtered[train_data_filtered.MISSING_DATA != True]

#### 0.5.5 OUTLIER
To handle the outliers, we look at statistical indicators and plot the boxplot.

In [ ]:
train_data_filtered.describe()

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=train_data_filtered[['N_COORDINATE_POINTS','TOTAL_FLIGHT_TIME_MINUTES','TOTAL_DISTANCE_KM']])
plt.show()

The cotinous attributes show a high number of outliers, with the number of coordinate points the widest spread.
To avoid loosing too much data, keeping the 95% quantile of the data regarding the N_COORDINATE_POINTS and TOTAL_DISTANCE seems to be the best choice. 

In [ ]:
train_data_filtered = train_data_filtered[(train_data_filtered.N_COORDINATE_POINTS <= train_data_filtered.N_COORDINATE_POINTS.quantile(0.95))
                                   & (train_data_filtered.TOTAL_DISTANCE_KM <= train_data_filtered.TOTAL_DISTANCE_KM.quantile(0.95))]

In [ ]:
plt.figure(figsize=(10, 10))
sns.boxplot(data=train_data_filtered[['N_COORDINATE_POINTS','TOTAL_FLIGHT_TIME_MINUTES','TOTAL_DISTANCE_KM']])
plt.show()

We can see some outliers remaining, however the spread is significantly reduced. Outliers in the test data will be kept to avoid too much reduction.

In [ ]:
sns.set()
plt.hist(train_data.TOTAL_FLIGHT_TIME_MINUTES[train_data.TOTAL_FLIGHT_TIME_MINUTES <= train_data.TOTAL_FLIGHT_TIME_MINUTES.quantile(0.95)], 
         label=f'Pre invalid trips N={train_data.shape[0]}')
plt.hist(train_data_filtered.TOTAL_FLIGHT_TIME_MINUTES[train_data_filtered.TOTAL_FLIGHT_TIME_MINUTES<= train_data_filtered.TOTAL_FLIGHT_TIME_MINUTES.quantile(0.95)], 
         label=f'Post invalid trips N={train_data_filtered.shape[0]}',
        alpha=0.5)
plt.title('Distribution - total flight time in minutes (95% quantile for visualization reasons)')
plt.legend()
plt.show()


In [ ]:
sns.set()
plt.hist(train_data_filtered.TOTAL_DISTANCE_KM[train_data_filtered.TOTAL_DISTANCE_KM<= train_data_filtered.TOTAL_DISTANCE_KM.quantile(0.95)], 
         label=f'Post invalid trips N={train_data_filtered.shape[0]}',
        alpha=0.5)
plt.title('Distribution - Count total distance (95% quantile for visualization reasons)')
plt.legend()
plt.show()

In [ ]:
sns.set()
plt.hist(train_data.N_COORDINATE_POINTS[train_data.N_COORDINATE_POINTS <= train_data.N_COORDINATE_POINTS.quantile(0.95)], 
         label=f'Pre invalid trips N={train_data.shape[0]}')
plt.hist(train_data_filtered.N_COORDINATE_POINTS[train_data_filtered.N_COORDINATE_POINTS<= train_data_filtered.N_COORDINATE_POINTS.quantile(0.95)], 
         label=f'Post invalid trips N={train_data_filtered.shape[0]}',
        alpha=0.5)
plt.title('Distribution - Count coordinate points (95% quantile for visualization reasons)')
plt.legend()
plt.show()

The reduction of the training data does not have a major effect on the data distribution. Optimization could be to compare performance with/without outliers 

In [ ]:
train_data = train_data_filtered.reset_index(drop=True)
test_data = test_data_filtered.reset_index(drop=True)

In [ ]:
perform_sanity_checks()

In [ ]:
train_data.TRIP_ID.value_counts()

In [ ]:
train_data[train_data.TRIP_ID == 1397172149620000454].POLYLINE_LIST.explode()

In [ ]:
train_data[train_data.TRIP_ID == 1397172149620000454]

#### 0.5.5 CALL_TYPE LOGIC

In [ ]:
def check_call_type(data):
    data_A = data[(data.CALL_TYPE == 'A') & (data.ORIGIN_CALL == np.NaN)]
    data_B = data[(data.CALL_TYPE == 'B') & (data.ORIGIN_STAND == np.NaN)]
    data_C = data[(data.CALL_TYPE == 'C') & (data.ORIGIN_STAND != np.NaN)].ORIGIN_STAND.nunique()
    return data_A, data_B, data_C

In [ ]:
check_call_type(train_data)

In [ ]:
check_call_type(test_data)

### 1. Exploratory Data Analysis  & Feature Engineering

In [ ]:
correlation_matrix = train_data.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
train_data.groupby('CALL_TYPE', as_index=False)[['TOTAL_FLIGHT_TIME_MINUTES', 'TOTAL_DISTANCE_KM']].agg(['mean', 'median','std'])

In [ ]:
train_data.groupby('WEATHER', as_index=False)[['TOTAL_FLIGHT_TIME_MINUTES', 'TOTAL_DISTANCE_KM']].agg(['mean', 'median','std'])

In [ ]:
train_data.groupby('WEATHER', as_index=False)['TRIP_ID'].nunique()

- No correlations in the data which are not obvious
- Especially surprising that the weather is not affecting the total flight time

In [ ]:
trip_freq = train_data.groupby('TAXI_ID', as_index=False)['TRIP_ID'].nunique().sort_values('TRIP_ID',ascending=False).reset_index(drop=True)

In [ ]:
trip_freq.TAXI_ID = trip_freq.TAXI_ID.astype(object)

In [ ]:
vc_origin_stand = train_data.ORIGIN_STAND.value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
vc_origin_stand

In [ ]:
sns.set()
plt.figure(figsize=(15,3))
plt.bar(vc_origin_stand.index,vc_origin_stand.values, label='TRIPs per ORIGIN STAND')
plt.xticks(vc_origin_stand.index, rotation=45)
plt.title(f'TRIPs per ORIGIN STAND N={vc_origin_stand.shape[0]}')
plt.show()

Several ORIGIN STAND have high demands, the high cardinality of the feature should be reduced by summarizing ORIGIN STANDS with few demands

In [ ]:
def extend_timestamps(data):
    data['TIMESTAMP_MONTH'] = pd.to_datetime(data['TIMESTAMP_DT']).dt.month
    data['TIMESTAMP_DAY'] = pd.to_datetime(data['TIMESTAMP_DT']).dt.day
    data['TIMESTAMP_WEEK'] = pd.to_datetime(data['TIMESTAMP_DT']).dt.isocalendar().week
    data['TIMESTAMP_YEAR'] = pd.to_datetime(data['TIMESTAMP_DT']).dt.year
    data['YEAR_MONTH'] = data['TIMESTAMP_YEAR'].astype(str) + '_' + data['TIMESTAMP_MONTH'].astype(str)
    return data

In [ ]:
train_data = extend_timestamps(train_data.copy())
test_data = extend_timestamps(test_data.copy())

In [ ]:
vc_year_month = train_data.YEAR_MONTH.value_counts().sort_index()

In [ ]:
sns.set()
plt.figure(figsize=(10,3))
plt.bar(vc_year_month.index, vc_year_month.values,label='Unique TRIPs per Month')
plt.xticks(vc_year_month.index, rotation=45)
plt.title(f'TRIPs per MONTH N={vc_year_month.shape[0]}')
plt.show()

#### 1.2 Feature Encoding

- CALL TYPE -> ONE_HOT ENCODING, no ordinal relationship
- WEATHER --> ONE HOT ENCODING, no ordinal relationship
- ORIGIN STAND --> Reduction of High cardinality + ONE HOT ENCODING
- MONTH/WEEK per year --> ONE HOT ENCODING or ORDINAL ENCODING

In [ ]:
def reduce_high_cardinality(data, columns=[]):
    for column_ in columns:
        vc = data[str(column_)].value_counts()
        median_freq = vc.median()
        majority_freq = vc[vc >= median_freq].index
        data[str(column_)+ '_agg'] = data.apply(lambda row: row[str(column_)] if row[str(column_)] in majority_freq else 'OTHER', axis=1)
        data[str(column_)+ '_agg'] = data[str(column_)+ '_agg'].astype(str)
    return data

def feature_encoding_oh(data, categories_oh):
    from sklearn.preprocessing import OneHotEncoder
    data_encoded = pd.DataFrame()
    for attribute_ in categories_oh:
        enc = OneHotEncoder()
        fenc = enc.fit_transform(X=data[str(attribute_)].values.reshape(-1,1)).toarray()
        df_fenc = pd.DataFrame(fenc, columns=enc.categories_)
        data_encoded = pd.concat([df_fenc, data_encoded], axis=1)
    return data_encoded
        

def feature_encoding_oe(data, categories_oe):
    from sklearn.preprocessing import OrdinalEncoder
    data_encoded = pd.DataFrame()
    for attribute_ in categories_oe:
        enc = OrdinalEncoder()
        fenc = enc.fit_transform(X=data[str(attribute_)].values.reshape(-1,1))
        df_fenc = pd.DataFrame(fenc, columns=[str(attribute_)+'_OE'])
        data_encoded = pd.concat([df_fenc, data_encoded], axis=1)
    return data_encoded

In [ ]:
train_data = reduce_high_cardinality(train_data, ['ORIGIN_STAND'])
test_data = reduce_high_cardinality(test_data, ['ORIGIN_STAND'])

In [ ]:
categories_oh = ['CALL_TYPE','WEATHER','ORIGIN_STAND_agg']
categories_oe = ['YEAR_MONTH']

In [ ]:
df_fenc_oh = feature_encoding_oh(train_data, categories_oh)
df_fenc_oe = feature_encoding_oe(train_data, categories_oe)
train_data = pd.concat([train_data, df_fenc_oh, df_fenc_oe],axis=1)

In [ ]:
df_fenc_oh = feature_encoding_oh(test_data, categories_oh)
df_fenc_oe = feature_encoding_oe(test_data, categories_oe)
test_data = pd.concat([test_data, df_fenc_oh, df_fenc_oe],axis=1)

In [ ]:
def add_binary_features(train_data, test_data):
    missing_features_test = train_data.columns.difference(test_data.columns)
    missing_features_train = test_data.columns.difference(train_data.columns)
    zero_data_test = np.zeros(shape=(test_data.shape[0],len(missing_features_test)))
    dummy_data_test = pd.DataFrame(zero_data_test, columns=missing_features_test)
    zero_data_train = np.zeros(shape=(train_data.shape[0],len(missing_features_train))) 
    dummy_data_train = pd.DataFrame(zero_data_train, columns=missing_features_train)
    
    return pd.concat([test_data, dummy_data_test],axis=1), pd.concat([train_data, dummy_data_train],axis=1)
    

In [ ]:
test_data, train_data = add_binary_features(train_data, test_data)

In [ ]:
assert(train_data.shape[1] == test_data.shape[1])

In [ ]:
non_features = ['CALL_TYPE','ORIGIN_CALL','ORIGIN_STAND', 'TOTAL_FLIGHT_TIME_SECONDS', 'START_POINT','DEST_POINT',
                'TIMESTAMP_MONTH','TIMESTAMP_DAY','TIMESTAMP_WEEK','TIMESTAMP_YEAR','YEAR_MONTH','ORIGIN_STAND_agg',
               'MISSING_DATA','POLYLINE','WEATHER','TAXI_ID','TIMESTAMP_DT','TIMESTAMP']

In [ ]:
train_data[['DEST_LON','DEST_LAT']] = pd.DataFrame(train_data.DEST_POINT.tolist(), columns=['DEST_LON','DEST_LAT'])
test_data[['DEST_LON','DEST_LAT']] = pd.DataFrame(test_data.DEST_POINT.tolist(), columns=['DEST_LON','DEST_LAT'])

In [ ]:
train_data[[column_ for column_ in train_data.columns if column_ not in non_features]].to_csv('s3://think-tank-casestudy/features_engineered/feature_engineered_regress_train.csv')
test_data[[column_ for column_ in train_data.columns if column_ not in non_features]].to_csv('s3://think-tank-casestudy/features_engineered/feature_engineered_regress_test.csv')

#### 1.3. Label encoding

1) Multi-regression output problem --> predict two labels: longitude and latitude

2) Re-write problem as multi-class problem through clustering
To define the classes, the last points per trip from the training data will be divided into clusters. When we have a large enough number of clusters, the clusters should be small enough to have their points close to their centroids. 

K-Means is a good approach as it minimizes the sum of squared distances within the cluster. Also it scales well to a large number of clusters. 

So the approach then predicts: **Probability of final destination of a trip being located in a specific cluster**

Steps:
- Get optimal number of clusters with help of interia
- Generate clusters with K-Means for each trip in training data
- Predict cluster for each trip in testing data
- Get centroids for each cluster 
- Calculate distance from points in cluster to centroid to evaluate

In [ ]:
##takes too long to execute
"""
from sklearn.cluster import KMeans
k_clusters = range(100,2000,100)
kmeans = [KMeans(n_clusters=n) for n in k_clusters]
X = train_data[['LAST_POINT_LONG','LAST_POINT_LAT']]
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
plt.plot(k_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Interia')
plt.title('Elbow Curve 100 to 2000 clusters ')
plt.show()
"""

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dest_lon_lat_scaled_train = scaler.fit_transform(train_data[['DEST_LON','DEST_LAT']])
dest_lon_lat_scaled_test = scaler.fit_transform(test_data[['DEST_LON','DEST_LAT']])


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1000)
kmeans.fit(dest_lon_lat_scaled_train)

In [ ]:
train_data['cluster_label'] = kmeans.labels_

In [ ]:
test_data['cluster_label'] = kmeans.predict(dest_lon_lat_scaled_test)

In [ ]:
cluster_data = pd.DataFrame(kmeans.cluster_centers_, columns=['center_longitude','center_latitude'])\
.reset_index().rename(columns={'index':'cluster_label'})

In [ ]:
train_data = pd.merge(train_data, cluster_data, how='left', on='cluster_label')

In [ ]:
train_data[[column_ for column_ in train_data.columns if column_ not in non_features]].to_csv('s3://think-tank-casestudy/features_engineered/feature_engineered_class_train.csv')
test_data[[column_ for column_ in train_data.columns if column_ not in non_features]].to_csv('s3://think-tank-casestudy/features_engineered/feature_engineered_class_test.csv')